# 用v1的model对v2预测 eval

In [1]:
import sys
root = '/home/featurize/data'
sys.path.append(f'{root}/pikachu/utils')
sys.path.append(f'{root}/pikachu/third')
sys.path.append(f'{root}/pikachu')
sys.path.append(f'{root}/pikachu/projects/ai/naic_seg')

In [2]:
import sys 
import os
import glob
import cv2
from PIL import Image
import numpy as np

from gezi import tqdm
from gezi.metrics.image.semantic_seg import Evaluator
import gezi

In [3]:
CLASSES = ['water', 'track', 'build', 'arable', 'grass', 'forest', 'bare', 'other']
NUM_CLASSES = len(CLASSES)
m = {}
for i in range(17):
  if i < 4:
    m[i + 1] = i
  else:
    m[i + 1] = i - 2

m2 = {
  0: 0,
  1: 1,
  2: 2,
  3: 1,
  4: 7,
  5: 7,
  6: 3,
  7: 3,
  8: 4,
  9: 4,
  10: 5,
  11: 5,
  12: 6,
  13: 6,
  14: 7,
}

In [16]:
def eval(pred_dir, label_dir='../input/eval.naic/label', from_v2=False):
#   key_metric = 'FWIoU'
  key_metric = 'IoU'

  pred_files = glob.glob(f'{pred_dir}/*')

  evaluator = Evaluator(CLASSES)
  class_mious = 0.

  t = tqdm(enumerate(pred_files), total=len(pred_files), ascii=True, desc= 'eval')
  for i, pred_file in t:
    file_name = os.path.basename(pred_file)
#     if int(file_name.split('.')[0]) != 8534:
#       continue
    evaluator2 = Evaluator(CLASSES)
    label_file = f'{label_dir}/{file_name}'
    label = np.asarray(Image.open(label_file))
    pred = cv2.imread(pred_file,cv2.IMREAD_UNCHANGED).astype(np.uint8)
    if from_v2:
      f = np.vectorize(lambda x: m[x])
      pred = f(pred).astype(np.uint8)
      f = np.vectorize(lambda x: m2[x])
      pred = f(pred).astype(np.uint8)
    
    f = np.vectorize(lambda x: m[x])
    label = f(label).astype(np.uint8)
    f = np.vectorize(lambda x: m2[x])
    label = f(label).astype(np.uint8)

    evaluator.eval_each(label[np.newaxis,:, :], pred[np.newaxis,:, :], metric=key_metric)
    evaluator2.eval_each(label[np.newaxis,:, :], pred[np.newaxis,:, :], metric=key_metric)

    binary_label = np.bincount(label.reshape(-1), minlength=NUM_CLASSES).astype(np.bool)
    binary_pred = np.bincount(pred.reshape(-1), minlength=NUM_CLASSES).astype(np.bool)
    intersections = binary_label * binary_pred
    intersection = np.sum(intersections, axis=-1)
    union = np.sum(binary_label, axis=-1) + np.sum(binary_pred, axis=-1) - intersection
    class_miou = np.mean(intersection / union)

    class_mious += class_miou

    metric = evaluator.eval_once(key_metric)
    metric2 = evaluator2.eval_once(key_metric)
    t.set_postfix({key_metric: metric2, 'IMAGE/CLASS/MIoU': class_mious / (i + 1)})
    
    if metric2 < 0.1:
      print(file_name, metric2, class_miou)

  res = evaluator.eval_once()
  res['IMAGE/CLASS/MIoU'] = class_mious / i
  gezi.set('evaluator', evaluator)
  
  return res

In [11]:
eval('../input/out.v1inferv2/')

73189.png 0.02232520692529596 0.42857142857142855
33928.png 0.046303006519588165 0.6666666666666666
97975.png 0.005286311711352897 0.2857142857142857
73317.png 0.029498704939439565 0.5
12654.png 0.013577881404189175 0.25
92216.png 0.08932820594439161 0.4
6538.png 0.0 0.0
3732.png 0.08069292491456967 0.6666666666666666
52984.png 0.07890410162905416 0.625
49580.png 0.0 0.0
44454.png 0.06163864967802132 0.25
70909.png 0.0718048095703125 0.2
99209.png 0.06904220581054688 0.125
26083.png 0.0 0.0
69505.png 0.04445503362312792 0.2857142857142857
95277.png 0.07061792975069779 0.375
77065.png 0.09200613839285714 0.14285714285714285
40556.png 0.0963291618258321 0.5714285714285714
37395.png 0.07931147183789083 0.2
41441.png 0.02862222089640524 0.6
2804.png 0.09613082010310144 0.5714285714285714
21992.png 0.0018280029296875 0.2
93185.png 0.08405968046488348 0.5
49707.png 0.094985654453715 0.3333333333333333
15022.png 0.06314817591250164 0.5
5745.png 0.08864109659187475 0.3333333333333333
8902.png 

KeyboardInterrupt: 

In [17]:
eval('../input/out/', from_v2=True)

25408.png 0.05145932341170585 0.25
25973.png 0.04288966712554897 0.6666666666666666
20781.png 0.02934229724817087 0.6
44170.png 0.021787114500046306 0.25
21402.png 0.07386297391543982 0.3333333333333333
8534.png 0.07723915215391472 0.4
18896.png 0.08671385997485109 0.5
40030.png 0.03744040263378231 0.6
15993.png 0.09835162993849972 0.5
14714.png 0.08015770676143558 0.75
629.png 0.010640066500415627 0.16666666666666666
36157.png 0.09550594877438905 0.75
3869.png 0.032909866048534395 0.6
8875.png 0.036997266907900006 0.5
14142.png 0.05447875324168762 0.75



{'FWIoU': 0.7912062081926461,
 'MIoU': 0.7890282091028283,
 'ACC/pixel': 0.882419921875,
 'ACC/class': 0.8773264806652123,
 'IoU/water': 0.7868756335792606,
 'IoU/track': 0.8004220188000045,
 'IoU/build': 0.8621505014199441,
 'IoU/arable': 0.8577092298412409,
 'IoU/grass': 0.6803549446697694,
 'IoU/forest': 0.7618928246547603,
 'IoU/bare': 0.7827567625984676,
 'IoU/other': 0.7800637572591792,
 'IMAGE/CLASS/MIoU': 0.9100152872430213}